In [1]:
import networkx as nx
import cenpy
import osmnx as ox
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib
from mpl_toolkits.axes_grid1 import make_axes_locatable
import contextily

C:\Users\peiyaozhao\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
ct = "Connecticut, USA"
ct_NewH = ['New Haven County, Connecticut, USA']
ct_Mids = ['Middlesex County, Connecticut, USA']
ct_Tola = ['Tolland County, Connecticut, USA']
ct_Hart = ['Hartford County, Connecticut, USA']
ct_Fair = ['Fairfield County, Connecticut, USA']
          #{'city':'Los Altos Hills', 'state':'California'},
          #'Loyola, California']
        
lyme = ['Lyme, Connecticut, USA']
Oldlyme = ['Old Lyme, Connecticut, USA']
Colchester = ['Colchester, Connecticut, USA']
NewMilford = ['New Milford, Connecticut, USA']
Bridgewater = ['Bridgewater, Connecticut, USA']
        
desired_building_types = ['school', 'commercial', 'retail','office','garage','hotel','church','train_station','dormitory',
                          'mosque','warehouse','stadium','clubhouse','chapel','fire_station','university','kindergarten',
                         'service','construction','hospital','plant_nursery','greenhouse','public','government','farm_auxiliary',
                         'farm','boathouse','sports_centre','dome','hangar','kiosk','college','napa auto parts','cathedral',
                         'synagogue','library','museum','convent','store','yes;commercial','university;yes','supermarket','parish_hall',
                          'motel','roof;commercial']

In [3]:
def footprint_squarefeet(place_name, building_types):
    graph = ox.graph_from_place(place_name, network_type='all')
    area = ox.geocode_to_gdf(place_name)
    bldgs = ox.geometries_from_place(place_name, tags={'building': building_types})

    building_geometries_series = gpd.GeoSeries(bldgs.geometry)
    total_footprint_area = building_geometries_series.area.sum()
    projected_crs = 'EPSG:6433'
    building_geometries_projected = bldgs.to_crs(projected_crs)
    total_footprint_area_projected = building_geometries_projected.area.sum()
    
    #print("Total Building Footprint Area:", total_footprint_area)
    #print("Total Building Footprint Area (Projected):", total_footprint_area_projected) #square feet
    return total_footprint_area_projected

In [4]:
ct = footprint_squarefeet(ct, desired_building_types)
NewH = footprint_squarefeet(ct_NewH, desired_building_types)
Mids = footprint_squarefeet(ct_Mids, desired_building_types)
Tola = footprint_squarefeet(ct_Tola, desired_building_types)
Hart = footprint_squarefeet(ct_Hart, desired_building_types)+footprint_squarefeet(lyme, desired_building_types)+footprint_squarefeet(Oldlyme, desired_building_types)+footprint_squarefeet(Colchester, desired_building_types)
Fair = footprint_squarefeet(ct_Fair, desired_building_types)+footprint_squarefeet(NewMilford, desired_building_types)+footprint_squarefeet(Bridgewater, desired_building_types)

C:\Users\peiyaozhao\AppData\Local\Temp\ipykernel_21076\4126494917.py:4: UserWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in a future release.
  bldgs = ox.geometries_from_place(place_name, tags={'building': building_types})
C:\Users\peiyaozhao\AppData\Local\Temp\ipykernel_21076\4126494917.py:7: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  total_footprint_area = building_geometries_series.area.sum()
C:\Users\peiyaozhao\AppData\Local\anaconda3\Lib\site-packages\pandas\core\frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
C:\Users\peiyaozhao\AppData\L

In [5]:
footprint = pd.DataFrame({
        'County': ['New Haven','Hartford','Middlesex','Tolland','Fairfield','Statewide'],
        'Total Footprint Area (square feet)': [NewH,Hart,Mids,Tola,Fair,ct]
    })

footprint['Ratio'] = footprint['Total Footprint Area (square feet)']/ct

ct_emissions = pd.read_excel('../../../data/raw/Public GHG data 1990-2021.xlsx', sheet_name="Emission Totals")

emi = ct_emissions.iloc[2,31]
footprint['GHG Emissions/MMTCO2e'] = footprint['Ratio']*emi
rows_to_sum = [1, 2, 3]

# Calculate the sum of the selected rows
sum_result = footprint.iloc[rows_to_sum].sum()
newH_footprint = footprint.iloc[0]
Fair_footprint = footprint.iloc[4]
state_footprint = footprint.iloc[5]

# Choose two rows by label (index)
rows_to_extract = [0, 4,5]

# Extract the selected rows into a new DataFrame
footprint_msa = footprint.loc[rows_to_extract].copy()

# Remove the selected rows from the original DataFrame
#footprint_msa = footprint.drop(rows_to_extract)
position_to_add = 2

# Concatenate the DataFrame before and after the new row
footprint_msa = pd.concat([footprint_msa.iloc[:position_to_add], pd.DataFrame([sum_result]), footprint_msa.iloc[position_to_add:]]).reset_index(drop=True)

footprint_msa.loc[0, 'County'] = 'New Haven-Milford'
footprint_msa.loc[1, 'County'] = 'Bridgeport-Stamford-Norwalk'
footprint_msa.loc[2, 'County'] = 'Hartford-East Hartford-Middletown'

new_row = {
    'County': 'Rest of the state',
    'Total Footprint Area (square feet)': footprint_msa.iloc[3]['Total Footprint Area (square feet)'] - footprint_msa.iloc[:3]['Total Footprint Area (square feet)'].sum(),
    'Ratio': footprint_msa.iloc[3]['Ratio'] - footprint_msa.iloc[:3]['Ratio'].sum(),
    'GHG Emissions/MMTCO2e': footprint_msa.iloc[3]['GHG Emissions/MMTCO2e'] - footprint_msa.iloc[:3]['GHG Emissions/MMTCO2e'].sum()
}

footprint_msa = pd.concat([footprint_msa.iloc[:3], pd.DataFrame([new_row]), footprint_msa.iloc[3:]]).reset_index(drop=True)

msa = footprint_msa.iloc[0:3,0]
values = footprint_msa.iloc[0:3,-1]
Footprint_ratio = footprint_msa.iloc[0:4,2]
categories = footprint_msa.iloc[0:4,1]

footprint_msa 

,County,Total Footprint Area (square feet),Ratio,GHG Emissions/MMTCO2e
0,New Haven-Milford,4.021517e+06,0.249936,0.955882
1,Bridgeport-Stamford-Norwalk,3.061254e+06,0.190256,0.727635
2,Hartford-East Hartford-Middletown,6.733395e+06,0.418479,1.600473
3,Rest of the state,2.274012e+06,0.141329,0.540514
4,Statewide,1.609018e+07,1.000000,3.824505


In [6]:
footprint_msa .iloc[1], footprint_msa .iloc[2] = footprint_msa .iloc[2].copy(), footprint_msa .iloc[1].copy()

In [7]:
footprint_msa 

,County,Total Footprint Area (square feet),Ratio,GHG Emissions/MMTCO2e
0,New Haven-Milford,4.021517e+06,0.249936,0.955882
1,Hartford-East Hartford-Middletown,6.733395e+06,0.418479,1.600473
2,Bridgeport-Stamford-Norwalk,3.061254e+06,0.190256,0.727635
3,Rest of the state,2.274012e+06,0.141329,0.540514
4,Statewide,1.609018e+07,1.000000,3.824505


In [8]:
footprint_msa.to_csv('../../../results/sectors/stationary-commercial-emissions.csv')